# Imports

In [ ]:
import joblib
import os
from skimage import data, color
from skimage.transform import resize
import numpy as np
import importlib
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#import sys
#sys.path.append('/content/drive/MyDrive/machine-learning/obligatorio/detector')

In [ ]:
import utils
importlib.reload(utils)
from utils import (
    browse_images,
    load_multiple_images,
    show_images,
    global_multiscale_detection,
    show_image_with_rectangle,
    compare_image_sets
)

#BASE_PATH = '/content/drive/MyDrive/machine-learning/obligatorio/detector'
BASE_PATH = ''

BATCH_SIZE = 1024
MAX_JOBS = 4
    
MODEL_NAME = 'model_x12_v1_150'
SCALER_NAME = 'scaler_x12_v1_150'
PCA_NAME = 'pca_x12_v1_150'

# Clasificador

In [ ]:
# Cargar modelo entrenado
clf = joblib.load(os.path.join(BASE_PATH, f'models/{MODEL_NAME}.joblib'))

# Cargar standard scaler
scaler = joblib.load(os.path.join(BASE_PATH, f'models/{SCALER_NAME}.joblib'))

# Cargar pca
pca = joblib.load(os.path.join(BASE_PATH, f'models/{PCA_NAME}.joblib'))

# Cargar Imagenes

## Agregar imágenes del directorio 'test_images'

In [ ]:
print("Explorando imágenes en test_images...")
available_images = browse_images(BASE_PATH)

In [ ]:
images_with_true_faces = load_multiple_images(os.path.join(BASE_PATH,"test_images"), available_images)

## Agregar astronauta

In [ ]:
astronaut = data.astronaut()
astronaut = color.rgb2gray(astronaut)
astronaut = resize(astronaut, (500, 500), anti_aliasing=True, preserve_range=True)
astronaut = (astronaut * 255).astype(np.uint8)
images_with_true_faces.append(('1-astronaut', astronaut, 1))

In [ ]:
show_images([item[1] for item in images_with_true_faces], max_per_row=4)

In [ ]:
# # Verificar
# for item in images_with_true_faces:
#     img = item[1]
#     print(f"{item[0]}")
#     print(f"  {img.shape}, Tipo: {img.dtype}, Rango: {img.min()} - {img.max()}, Rostros: {item[2]}")

In [ ]:
filename_list = [item[0] for item in images_with_true_faces]
image_list = [item[1] for item in images_with_true_faces]
true_faces_list = [item[2] for item in images_with_true_faces]

# Correr detector

## Params

In [ ]:
test_scales = np.arange(0.5, 5.0, 0.5)

threshold = 0.9
overlap_thresh = 0.05
step = 4

hog_params = {
    # "orientations": 9,
    # "pixels_per_cell": (8, 8),
    # "cells_per_block": (2, 2),
    # "block_norm": "L2-Hys",
    # "visualize": False,
    # "transform_sqrt": True,
    "feature_vector": True
}

## Detector sobre 1 imágen

### Mostrar imágen con rectángulo para estimar tamaño del scale

In [ ]:
# Elegir la imagen a procesar
image_index = 1
# Estimar un true scale
true_scale = 3.5
# Mostrar imagen con rectángulo para estimar tamaño del scale
show_image_with_rectangle(image_list[image_index], true_scale)

In [ ]:
global_multiscale_detection(
    image_list[image_index],
    clf,
    scaler,
    pca,
    test_scales=[true_scale],
    threshold=threshold,
    overlap_thresh=overlap_thresh,
    batch_size=BATCH_SIZE,
    max_jobs=MAX_JOBS,
    hog_params=hog_params,
    plot=True,
    save_result=False,
    result_dir=f"detections/{MODEL_NAME}",
    filename=filename_list[image_index]
)

### Multiscale detection

In [ ]:
global_multiscale_detection(
    image_list[image_index],
    clf,
    scaler,
    pca,
    test_scales=test_scales,
    threshold=threshold,
    overlap_thresh=overlap_thresh,
    batch_size=BATCH_SIZE,
    max_jobs=MAX_JOBS,
    hog_params=hog_params,
    plot=True,
    save_result=False,
    result_dir=f"detections/{MODEL_NAME}",
    filename=filename_list[image_index]
)

## Detector sobre todo el dataset

In [ ]:
# For each image, perform multiscale detection
for image_index, (filename, img, num_faces) in enumerate(images_with_true_faces):
    global_multiscale_detection(
        img,
        clf,
        scaler,
        pca,
        test_scales=test_scales,
        threshold=threshold,
        overlap_thresh=overlap_thresh,
        step=step,
        batch_size=BATCH_SIZE,
        max_jobs=MAX_JOBS,
        hog_params=hog_params,
        plot=False,
        save_result=True,
        result_dir=f"detections/{MODEL_NAME}",
        filename=filename
    )
print(f"Detección de rostros completada. Resultados guardados en 'detections/{MODEL_NAME}'.")

In [ ]:
dir_A = "detections/model_x2_v2_50_2step"           
dir_B = "detections/model_x12_v1_150_4step"
dir_C = "detections/model_x24_v1_150_4step"
compare_image_sets(dir_A, dir_B, dir_C)